# Scala implicits

Implicits in Scala are also pretty ubiquous. For instance, we can find them in the Scala standard library, and two popular frameworks: akka streams and Spark.

![futureapi](../misc/futureapi.png)

![futureapply](../misc/futureapply.png)

![futurefoldleft](../misc/futurefoldleft.png)

![akka-flow](../misc/akka-api-flow.png)

![akka-flow-runwith](../misc/akka-api-flow-runwith.png)

![spark-dataset](../misc/spark-api-dataset.png)

![dataset-flatmap](../misc/spark-api-dataset-flatmap.png)

![spark-groupbykey](../misc/spark-api-dataset-groupbykey.png)

# Implicits as term inference

We already saw that the Scala compiler can infer type parameters in calls to generic functions. For instance, given:

we may call our function passing explicitly the type parameter:

or we may omit the type parameter and let the Scala compiler to infer its proper value:

Can _values_ also be inferred by the Scala compiler? Yes, they can! For instance, in order to create an asynchronous computation using Scala `Future`s, we need to pass its `ExecutionContext`, i.e. the component that will be in charge of executing it. This is the signature of the factory method: 

![futureapply](../misc/futureapply.png)

As you can see, the execution context parameter is maked as `implicit` which means that the Scala compiler, in principle, can infer the value of that parameter for us. Of course, we can also pass the parameter explicitly:

In [ ]:
import scala.concurrent.{Future, ExecutionContext}



but the idiomatic way to write this invocation is as follows:

Ooops. We forgot an important detail. In order for Scala to infer the type parameter we didn't have to do anything: Scala infers the type `Int` just from the type of the code to be executed. However, in order to infer the execution context, Scala needs some help in the form of `implicit` declarations: 

In [ ]:
val ec: ExecutionContext = ExecutionContext.global



In sum, if we want Scala to infer the value of some parameter we mark it as `implicit`. If we want some value to be used as the implicit parameter of a given type, we declare that value as `implicit`. But be careful not to declare more than one implicit value for a given type, at least at the same priority level. Otherwise, Scala complains of ambiguity. For instance:

In [ ]:
val ec2: ExecutionContext = ExecutionContext.global



As we mentioned in passing, Scala looks for implicits at severals levels of priority. You can find [here](https://docs.scala-lang.org/tutorials/FAQ/finding-implicits.html) a detailed explanation of these levels, which, besides the current scope, include companion objects of the type (and subtypes involved), ancestors of companion objects, etc. For instance, since implicits found in super-classes have less priority the following example compiles:

In [8]:
object Scope3{
    import Scope1._
    implicit val ec2: ExecutionContext = ExecutionContext.global

    Future(1)        
}

defined object Scope3

# Implicits for context parameters

The previous example shows a very common use case of implicit parameters: to inject runtime dependencies, i.e. contextual parameters that are needed in order to execute our code. But why is this useful at all? Let's pretend that implicits are not available, and see what happens with the following example:

In [ ]:
def foo(a: Int): Boolean = 
    bar(a).length > 0

def bar(a: Int): String = 
    a.toString

There are two major kinds of parameters in this dummy example: parameters `a` and `b`, and the execution context `ec`. These are very different in the sense that the former ones pertain to the business logic (so to speak) of the function, whereas the latter is concerned with a different aspect altogether (execution, in particular) and will be typically passed through unchanged. 

Let's see what happens when we add implicits:

In [ ]:
def foo(a: Int)(ec: ExecutionContext): Future[Boolean] = 
    bar(a)(ec).map(_.length > 0)(ec)

def bar(a: Int)(ec: ExecutionContext): Future[String] = 
    Future(a.toString)(ec)

As you can see, the programmer is no longer in charge of the repetitive task of passing the execution context: the compiler does that for her. This means more concise code and better separation of concerns. Rightly, the signatures still refer to the `ExecutionContext`, which means that this decoupling between logic and interpration is not fully completed. Fortunately, Scala 3 will feature *implicit function types*, which help significantly in this regard. Alternatively, we can also embrace the `Reader` monad ... but this is for other session.

### Conditional implicit values

We find a similar use implicits for injecting context paremeters in Akka stream. Let's analyse this example with some more detail.

In [ ]:
import $ivy.`com.typesafe.akka::akka-stream:2.6.4`
import java.time._
import scala.concurrent._, duration._
import akka._
import akka.actor._
import akka.stream._
import akka.stream.scaladsl._

repl.pprinter() = repl.pprinter().copy(defaultHeight = 5 )

Let's say that we have this pipeline that simply generates a stream of integer values which are then printed in the console:

This is just a declarative program. If we want our pipeline to be executed we need to `run` it:

Oooops (again). In order to run a pipeline we need a `Materializer`: the component which will be in charge of executing the pipeline (it thus plays a similar role to the `ExecutionContext` of future-based computations). This is explicitly stated in the `run` signature:

![run](../misc/runnablegraph.png)

![runnablegraphrun](../misc/runnablegraphrun.png)

The standard `Materializer` that is provided by the akka stream library executes pipelines by means of an actor system. So, we first need to create an `ActorSystem`:

and we can now run our pipeline: 

But, wait, `run` needs an implicit value of the type `Materializer`, and we defined an implicit value of type `ActorSystem`. How did Scala infer an implicit value for the former type? The answer lies in the following declaration located in the companion object of the `Materializer` class: 

In [ ]:
/*

object Materializer {

  implicit def matFromSystem(
      implicit provider: ClassicActorSystemProvider): Materializer =
    ???

}
*/

`matFromSystem` is called a _conditional implicit value_, meaning that Scala can construct an implicit value for the `Materializer` type, _provided that_ it can infer an implicit value for a `ClassicActorSystemProvider`. In our example, it certainly can, since we declared an implicit value of the type `ActorSystem` (which extends the trait `ClassicActorSystemProvider`). Ok, but how did the Scala compiler find that conditional implicit value?, because it was certainly not in scope. Right, but `matFromSystem` is defined in the _companion object_ of the `Materializer` type, and we already said Scala looks automatically for implicits there, among other places.


# Implicits for extension methods

Another common use case for implicits is the implementation of extension methods: there is a class for which we have no control, or no interest in modifying its definition, and, yet, we would like to add some new methods. The Scala standard library of Scala has good examples of this pattern. For instance, as we already know, the `String` class comes actually from Java:

and `java.lang.String` does not declares a `map` method. How then can we make this call?

The answer lies in the following declaration in the `Predef.scala` file:

In [ ]:
/*
  implicit def augmentString(x: String): StringOps = 
      new StringOps(x)
*/

This is not a conditional implicit value, because the argument `x` is not implicit. This is a so-called *implicit conversion* that allows the Scala compiler to convert a `String` into an object of type `StringOps`. This class is the one that actually provides the new method `map`. It looks like this:

Now, in order to extend the `String` class with this new method, we have to do something similar to what the Scala library does, i.e. provide an implicit conversion:

And now it works!

This pattern is so common that Scala 2.10 provided a special construct for it: *implicit classes* (you may also think of implicit classes in terms of the adapter OO pattern).

In [ ]:
class MyStringOps(x: String){
    def mymap(f: Char => Char): String = 
        "dummy"
}

Note that we didn't need to write the implicit conversion function. Also, note that that we changed the name of the method. This is in order to avoid a compilation error due to multiple conversions available (a problem of _ambiguity_).

# Implicits for ad-hoc polymorphism

Let's come back to the use of implicits in the following signature of the Spark framework:

![spark-dataset](../misc/spark-api-dataset.png)

![dataset-flatmap](../misc/datasetmap.png)

Very briefly, objects of the class `Dataset` represent distributed dataset transformation programs. In this context, an `Encoder[T]` is used to convert JVM objects of type `T` from/to `Row`s, the dynamically typed internal representation of Spark. The definition of the `Encoder[T]` API does not mention, however, these conversion functions:

In [ ]:
/*
trait Encoder[T] extends Serializable {

  /** Returns the schema of encoding this type of object as a Row. */
  def schema: StructType

  /**
   * A ClassTag that can be used to construct and Array to contain a collection of `T`.
   */
  def clsTag: ClassTag[T]
}
*/

Technically, the trait `Encoder[T]` defines a _type class_, namely the class of types that can be serialized/deserialized to/from Spark `Row`s.

The first difference with the use case of context parameters (e.g. the `ExecutionContext` in futures) is that the implicit parameter `Encoder[U]` is related to the generic parameter `U`. What does this mean? Given that the signature is generic, we may intend to apply this method to any type `U`. However, this will only be possible if type `U` is internally supported by Spark, i.e. if there exists an implicit instance of the `Encoder` class for the type `U`. Therefore, this method is not really a case of parametric polymorphism, where there are no constraints over the type parameter. And it is not a case of subtype plymorphism either, since there are no subtype bounds on `U`. This type of polymorphism is named _ad-hoc polymorphism_ since the implementation depends on the particular, _ad-hoc_, implementation of the trait `Encoder` for the type `U`. 

Scala offers some syntactic sugar to express this kind of polymorphism more succintly: so-called, _context bounds_. Using this feature, the `map` signature is written as follows:

In [ ]:
/*
class Dataset[T]{
 def map[U : Encoder](func: T => U): Dataset[U] = 
   ...
}
*/

It's actually debatable if the `Encoder` example is a clear-cut case of ad-hoc polymorphism, since the encoder evidence does not affect the behaviour of the `map` method, but of the `Dataset` interpreters (actions, in particular). What we are actually doing is enforcing a constraint in the `Dataset` API, i.e. something nearer to the use case of implicits for checking type-level proofs.

A typical use case for type classes is the following one: we want to aggregate elements of a list according to the specific, ad-hoc, aggregators of the different types. We may start from this signature:

but, clearly, we know nothing about `T`, in particular, about its aggregator. So, we extend the signature with the parameters needed: 

In [ ]:
def aggregate(elements: List[Int]): Int = 
    elements match {
        case Nil => 0
        case head :: tail => aggregate(tail) + head 
    }

and, we can now, aggregate integers, strings, and whatever: 

Type classes come into play when we decide to pack the extra parameters in a special-purpose module, namely, `Aggregatable`, the class of types whose values can be aggregated: 

And, now, we can write our signature as follows:

In [ ]:
def aggregate[T](elements: List[T])(zero: T, combine: (T, T) => T): T =
    elements match {
        case Nil => zero : T
        case head :: tail => combine(head, aggregate(tail)(zero, combine))
    }

or, using context bounds: 

As you can see, the signature is clearer now, but the body got somewhat polluted with `implicitly` calls. We can remove that noise with special-purpose syntax (using implicit classes). Last, the `Aggregatable` type class is, of course, the `Monoid` type class.

What did we gain using type classes instead of plain function parameters? 
* More concise syntax at call sites
* Improved reusability of instances
* Improved testability (type classes laws)

# Looking further

Other implicits-related stuff we didn't have time to cover:
* Type-level functions (doing type-level computing à la shapeless)
* Standard type-level proofs (`<:<`, `=:=`, conversions, etc.)
* Implicits in dotty (better separation of intent vs. mecanism, implicit function types,  special syntax, ...)


# Takeaways

* First, think of your own use case. Don't use implicits without a clear-cut intent. 
* Exploit implicits for injecting run-time dependencies (i.e. context)
* Exploit implicits whenever the adapter pattern comes to your mind 
* Exploit implicits for ad-hoc polymorphism (and avoid inheritance!)
* Look forward to Scala 3.0 enhancements in regard to implicits
* Be aware of cons in Scala 2.x (e.g. compilation times, misuse, ...)

![downs](../misc/downtimes.png)

# References

FILIP KŘIKAVA, HEATHER MILLER, JAN VITEK. [Scala Implicits Are Everywhere](https://arxiv.org/pdf/1908.07883.pdf)

MARTIN ODERSKY. [Simplicitly: Foundations and Applications of Implicit Function Types](https://popl18.sigplan.org/details/POPL-2018-papers/85/Simplicitly-Foundations-and-Applications-of-Implicit-Function-Types)

[Implicits in Dotty](https://dotty.epfl.ch/docs/reference/contextual/motivation.html)